# Check aggregate entities

* 

In [8]:
import plotly.io as pio
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.express as px

load_dotenv()

pio.renderers.default = 'notebook'

In [9]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-goerli-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [10]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [11]:
agg_query = gql("""
query agg(
    $last_id: ID!
  ) {
  futuresAggregateStats(
    where: {asset: "0x", period: "86400", id_gt: $last_id}
  ) {
    id
    marketKey
    asset
    period
    timestamp
    feesSynthetix
    feesKwenta
    feesCrossMarginAccounts
    trades
    volume
  }
}
""")


### Run a query

In [12]:
agg_params = {
    'last_id': ''
}

agg_cols = [
    'feesSynthetix',
    'feesKwenta',
    'feesCrossMarginAccounts',
    'volume'
]

agg_bytes_cols = [
    'asset',
    'marketKey'
]

df_agg = await run_recursive_query(agg_query, agg_params, 'futuresAggregateStats')
df_agg = df_agg.replace({None: np.nan})
df_agg = clean_df(df_agg, decimal_cols=agg_cols, bytes_cols=agg_bytes_cols)


In [25]:
df_agg.head()


,id,marketKey,asset,period,timestamp,feesSynthetix,feesKwenta,feesCrossMarginAccounts,trades,volume
0,1671148800-86400-0x,,,86400,1671148800,218.06225081677274515,241.11283037281301696,0,45,44172.568240081902551739
1,1671235200-86400-0x,,,86400,1671235200,5.248698078708612748,9.320506517960022853,0,8,6459.631161627221114229
2,1671321600-86400-0x,,,86400,1671321600,1001.830587872446117638,1003.830054806870231894,0,3,1002837.002482530637180817
3,1671408000-86400-0x,,,86400,1671408000,1501.626591283797017249,1532.626591283797017249,0,31,1998162.533156205839209719
4,1671494400-86400-0x,,,86400,1671494400,631.054134706987739574,645.890045520080297055,0,29,1152087.729635435956517604


In [14]:
agg_params = {
    'last_id': ''
}

agg_cols = [
    'feesSynthetix',
    'feesKwenta',
    'feesCrossMarginAccounts',
    'volume'
]

agg_bytes_cols = [
    'asset',
    'marketKey'
]

df_agg_compare = await run_recursive_query(agg_query, agg_params, 'futuresAggregateStats', endpoint=OLD_SUBGRAPH_ENDPOINT)
df_agg_compare = df_agg_compare.replace({None: np.nan})
df_agg_compare = clean_df(df_agg_compare, decimal_cols=agg_cols, bytes_cols=agg_bytes_cols)


In [26]:
df_agg_compare.head()


,id,marketKey,asset,period,timestamp,feesSynthetix,feesKwenta,feesCrossMarginAccounts,trades,volume
0,1671148800-86400-0x,,,86400,1671148800,218.06225081677274515,0,0,45,44172.568240081902551739
1,1671235200-86400-0x,,,86400,1671235200,5.248698078708612748,0,0,8,6459.631161627221114229
2,1671321600-86400-0x,,,86400,1671321600,1001.830587872446117638,0,0,3,1002837.002482530637180817
3,1671408000-86400-0x,,,86400,1671408000,1501.626591283797017249,0,0,31,1998162.533156205839209719
4,1671494400-86400-0x,,,86400,1671494400,631.054134706987739574,0,0,29,1152087.729635435956517604


## Compare them

In [20]:
compare_cols = ['id', 'period', 'timestamp', 'feesSynthetix', 'feesKwenta', 'trades', 'volume']
df_agg[compare_cols].merge(
    df_agg_compare[compare_cols],
    on='id',
    suffixes=['_new', '_old']
).head()

,id,period_new,timestamp_new,feesSynthetix_new,feesKwenta_new,trades_new,volume_new,period_old,timestamp_old,feesSynthetix_old,feesKwenta_old,trades_old,volume_old
0,1671148800-86400-0x,86400,1671148800,218.06225081677274515,241.11283037281301696,45,44172.568240081902551739,86400,1671148800,218.06225081677274515,0,45,44172.568240081902551739
1,1671235200-86400-0x,86400,1671235200,5.248698078708612748,9.320506517960022853,8,6459.631161627221114229,86400,1671235200,5.248698078708612748,0,8,6459.631161627221114229
2,1671321600-86400-0x,86400,1671321600,1001.830587872446117638,1003.830054806870231894,3,1002837.002482530637180817,86400,1671321600,1001.830587872446117638,0,3,1002837.002482530637180817
3,1671408000-86400-0x,86400,1671408000,1501.626591283797017249,1532.626591283797017249,31,1998162.533156205839209719,86400,1671408000,1501.626591283797017249,0,31,1998162.533156205839209719
4,1671494400-86400-0x,86400,1671494400,631.054134706987739574,645.890045520080297055,29,1152087.729635435956517604,86400,1671494400,631.054134706987739574,0,29,1152087.729635435956517604


In [17]:
df_agg.shape

(47, 10)

In [18]:
df_agg_compare.shape

(47, 10)

In [23]:
compare_cols = ['id', 'period', 'timestamp', 'feesSynthetix', 'feesKwenta', 'trades', 'volume']
df_agg[compare_cols].equals(df_agg_compare[compare_cols])

False

In [24]:
compare_cols = ['id', 'period', 'timestamp', 'feesSynthetix', 'trades', 'volume']
df_agg[compare_cols].equals(df_agg_compare[compare_cols])

True